# A score and skill keeping system in python

- Situation
  - playing competitive games
  - multiplayer, various pairings
  - determine who's better than who
  - create fair pairings


- Idea
  - keep track of who wins
  - calculate skill, compare


- Extension
  - show individual developments

## Architecture

![Architecture](architecture.png)

## Score keeping: Domain specific language & Wisent package

- Requirements
  - easy way of writing down scores
  - first place, second place, etc.


- Idea
  - develop a custom domain specific language ([wiki](https://en.wikipedia.org/wiki/Domain-specific_language)) / grammar 
  - parse that grammar using wisent 


- Restrictions
  - context-free grammar, left-to-right, deterministic

### Grammar in use: game_scores.txt

- examples how to write down game scores

```text
# 1 vs 1
Foo > Bar
Bar > Foo
Bar = Foo

# Free for all
Foo > Bar > Batz
Foo > Bar = Batz
Foo = Bar > Batz
Foo = Bar = Batz

# Teams
(Foo, Bar) > (Batz, Foobar)
Foo > (Bar, Batz)
```

### TrueSkill needs

```python
# Foo = (Bar, Batz) > Foobar
teams = [[player_foo], [player_bar, player_batz], [player_foobar]]
ranks = [1, 1, 2]
```

### Grammar definition: grammar.wi

- Rules
- Symbols, transparent symbols

```text
game: _winorder ;

_winorder: leftwins | draw | _team ;
leftwins: _team '>' _winorder ;
draw: _team '=' _winorder ;

_team: brackets | PLAYER ;
brackets: '(' _multiple ')' ;
_multiple: PLAYER | player_in_team ;
player_in_team: PLAYER ',' _multiple ;
```

In [5]:
from wisentparser import Parser
from calc_skills import tokenize, eval_tree

In [6]:
game_line = "Foo > Bar"
tokens = tokenize(game_line); tokens

[('PLAYER', 'Foo'), ('>',), ('PLAYER', 'Bar')]

In [7]:
p = Parser(); tree = p.parse(tokens); tree

('game', ('leftwins', ('PLAYER', 'Foo'), ('>',), ('PLAYER', 'Bar')))

```python
def eval_tree(tree):
# ...
    if tree[0] == 'leftwins':
        left = eval_tree(tree[1])
        right = eval_tree(tree[3])
        res_teams = left["teams"] + right["teams"]
        res_ranks = left["ranks"] + [
            rank + max(left["ranks"]) for rank in right["ranks"]]
        return {"teams": res_teams, "ranks": res_ranks}
# ...
```

In [8]:
eval_tree(tree)

{'ranks': [1, 2], 'teams': [['Foo'], ['Bar']]}

## Skill calculation: TrueSkill package

work in progress

## Visualization: 

work in progress

## References

### Trueskill
- [TrueSkill project](http://trueskill.org/)
- [TrueSkill on GitHub](https://github.com/sublee/trueskill)
- [TrueSkill on Wikipedia](https://en.wikipedia.org/wiki/TrueSkill)
- [TrueSkill explanation](http://bit.ly/computing-your-skill)
- [TrueSkill paper](http://research.microsoft.com/apps/pubs/default.aspx?id=67956)

### Wisent
- [Wisent author's project](http://www.seehuhn.de/pages/wisent)
- [Wisent on GitHub](https://github.com/seehuhn/wisent)
- [Wisent manual](http://www.seehuhn.de/manuals/wisent/)
- [Wisent tutorial]()

### References

### Visualization
not yet done

### This talk
- This talk and project: [https://github.com/data-raccoon/trueskill_ranking](https://github.com/data-raccoon/trueskill_ranking)
- Slides: [Jupyter](http://jupyter.org/)
- Architecture graph: [yEd Graph Editor](https://www.yworks.com/products/yed)